# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686575


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:42,  1.53s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:50,  1.87s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:47,  1.83s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:34,  1.40s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:14<00:01,  2.89it/s]

Rendering models:  90%|█████████ | 27/30 [00:14<00:00,  3.03it/s]

Rendering models:  93%|█████████▎| 28/30 [00:14<00:00,  3.15it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  3.70it/s]

BridieK                               0.000484
not-logged-in-66525592fb07ceea3915    0.002024
spaceguy117                           0.001389
not-logged-in-20a275b53abec8db9f41    0.006362
shirogane                             0.000327
not-logged-in-f7e2260ed4a9517a3e4a    0.001037
amber_9                               0.002013
Camelron                              0.000262
Super_Novah                           0.000277
not-logged-in-2159616075e1803d6325    0.000885
not-logged-in-7c0e3b1ae8dc7fbb6b66    0.001894
mmcdonough4                           0.000697
Windstar187                           0.443733
not-logged-in-5f30b2bb9b7f9d49da41    0.010096
AlexYoung35                           0.000415
TRISTANM18                            0.000664
not-logged-in-314e2633afbe4d985ce9    0.001187
acapirala                             0.000305
not-logged-in-2c3093628ea1c6e967f7    0.000864
munschy                               0.002242
Lavadude                              0.178363
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())